# Modelling NIQS Hardware TF

In [1]:
import sys
sys.path.insert(0, '../../src_numpy/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import torch.optim as optim
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from cost_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from src_torch import *


#from src_tf import *

In [2]:
def apply_map(state, kraus_list):
    state = [K@state@tf.linalg.adjoint(K) for K in kraus_list]
    state = tf.math.reduce_sum(tf.stack(state), axis=0)
    return state


def expectation_value(state, observable):
    ev = tf.linalg.trace(observable@state)
    return ev

In [21]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

n = 5
d = 2**n
rank = 2

A = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
B = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
G = A + 1j*B
Q, R = tf.linalg.qr(G, full_matrices = False)
D = tf.linalg.tensor_diag_part(R)
D = tf.math.sign(D)
D = tf.linalg.diag(D)
U = Q@D


kraus_target_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]

In [22]:
N = 1000
state_index, observ_index = index_generator(n, N, trace=False)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):

    config = numberToBase(i, 6, n)
    state = tf.convert_to_tensor(prepare_input(config), dtype = tf.complex64)

    config = numberToBase(j, 3, n)
    observable = tf.convert_to_tensor(pauli_observable(config),  dtype = tf.complex64)
    
    input_list.append([state, observable])

target_list = [expectation_value(apply_map(input[0], kraus_target_list), input[1]) for input in input_list]
optimizer = tf.optimizers.Adam(learning_rate=0.05)

A = tf.Variable(tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64), trainable=True)
B = tf.Variable(tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64), trainable=True)

In [23]:
for i in range(200):
    with tf.GradientTape() as tape:
        G = A + 1j*B
        Q, R = tf.linalg.qr(G, full_matrices = False)
        D = tf.linalg.tensor_diag_part(R)
        D = tf.math.sign(D)
        D = tf.linalg.diag(D)
        U = Q@D

        kraus_model_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]
        pred_list = [expectation_value(apply_map(input[0], kraus_model_list), input[1]) for input in input_list]
        loss = tf.math.reduce_mean(tf.stack([(target - predicted)**2 for target, predicted in zip(target_list, pred_list)]))

    grads = tape.gradient(loss, [A,B])
    optimizer.apply_gradients(zip(grads, [A, B]))
    
    print(f"loss: {loss: .4f}, {tf.math.reduce_mean(tf.math.abs(G))}")

loss:  0.0311+0.0000j, 1.2708005905151367
loss:  0.0250+0.0000j, 1.2718511819839478
loss:  0.0199-0.0000j, 1.2739124298095703
loss:  0.0159+0.0000j, 1.2768845558166504
loss:  0.0127+0.0000j, 1.281261682510376
loss:  0.0104+0.0000j, 1.2870829105377197
loss:  0.0085+0.0000j, 1.2943997383117676
loss:  0.0071+0.0000j, 1.3033344745635986
loss:  0.0060+0.0000j, 1.313098669052124
loss:  0.0051+0.0000j, 1.3233588933944702
loss:  0.0044+0.0000j, 1.334221363067627
loss:  0.0038+0.0000j, 1.3453220129013062
loss:  0.0034+0.0000j, 1.356780767440796
loss:  0.0030+0.0000j, 1.36844801902771
loss:  0.0026+0.0000j, 1.3801813125610352
loss:  0.0023+0.0000j, 1.3918933868408203
loss:  0.0021-0.0000j, 1.4030563831329346
loss:  0.0018+0.0000j, 1.413879156112671
loss:  0.0017+0.0000j, 1.4244400262832642
loss:  0.0015+0.0000j, 1.434734582901001
loss:  0.0013+0.0000j, 1.4446110725402832
loss:  0.0012+0.0000j, 1.454108476638794
loss:  0.0011+0.0000j, 1.4632070064544678
loss:  0.0010+0.0000j, 1.4719059467315674


KeyboardInterrupt: 

In [25]:
my_list = [1,2,3,4,5]

In [28]:
my_list[0,1]

TypeError: list indices must be integers or slices, not tuple